# Teaching a Deep-Q-Learning Agent to Play Tetris

This notebook provides you with a complete code example that trains a train a deep reinforcement learning model to play Tetris.

## Implementing a Simplified Tetris

In [ ]:
import numpy as np
import random

class Tetris:
    """Simplified Tetris."""
    
    TILES = [
        [
            [[0, 2]],  # Tile 0, orientation 0.
            [[0, 1], [0, 1]],  # Tile 0, orientation 1.
        ],
        [
            [[0, 1], [1, 2]],  # Tile 1, orientation 0.
            [[1, 2], [0, 1]],  # Tile 1, orientation 1.
        ],
        [
            [[0, 2], [1, 2]],  # Tile 2, orientation 0.
            [[0, 2], [0, 1]],  # Tile 2, orientation 1.
            [[0, 1], [0, 2]],  # Tile 2, orientation 2.
            [[1, 2], [0, 2]],  # Tile 2, orientation 3.
        ],
        [
            [[0, 2], [0, 2]],  # Tile 3, orientation 0.
        ],
    ]
    
    UNDEFINED = -1

    def __init__(self, rows, cols, max_tiles, random_seed):
        """Initialize Tetris."""
        self.rows, self.cols = rows, cols
        self.max_tiles = max_tiles
        self.random_seed = random_seed

        self.restart()

    def restart(self):
        """Restart the game."""
        self.board = np.full((self.rows, self.cols), Tetris.UNDEFINED)
        
        self.current_tile = Tetris.UNDEFINED
        self.tile_x = Tetris.UNDEFINED
        self.tile_y = Tetris.UNDEFINED
        self.tile_orientation = Tetris.UNDEFINED
        
        self.gameover = False
        self.tile_count = 0
        
        # Create predefined tile sequence.
        rand_state = random.getstate()
        random.seed(self.random_seed)
        self.tile_sequence = [random.randint(0, len(Tetris.TILES) - 1)
                              for x in range(self.max_tiles)]
        random.setstate(rand_state)
        
        self.reward = 0
                
        self.next_tile()
        
    def next_tile(self):
        """Get the next tile."""
        if self.tile_count < self.max_tiles:
            if self.random_seed is not None:
                self.current_tile = self.tile_sequence[self.tile_count]
            else:
                self.current_tile = random.randint(0, len(Tetris.TILES) - 1)
            self.tile_x = self.cols // 2
            self.tile_y = self.rows
            self.tile_orientation = 0
        
            self.tile_count += 1
        else:
            self.gameover = True
    
    def move_left(self):
        """Move current tile to the left."""
        if self.tile_x - 1 >= 0:
            self.tile_x -= 1
            return True
        else:
            return False
    
    def move_right(self):
        """Move current tile to the right."""
        tile_width = len(Tetris.TILES[self.current_tile][self.tile_orientation])
        if self.tile_x + 1 <= self.cols - tile_width:
            self.tile_x += 1
            return True
        else:
            return False
    
    def rotate(self):
        """Rotate current tile."""
        new_orientation = ((self.tile_orientation + 1) 
                           % len(Tetris.TILES[self.current_tile]))
        tile_width = len(Tetris.TILES[self.current_tile][new_orientation])
        if self.tile_x <= self.cols - tile_width:
            self.tile_orientation = new_orientation
            return True
        else:
            return False
        
    def drop(self):
        """Drop current tile and update game board."""
        tile = Tetris.TILES[self.current_tile][self.tile_orientation]
        
        # Find first location where the tile collides with occupied locations.
        self.tile_y = 0
        for x in range(len(tile)):
            cury = -1
            for y in range(self.rows -1, -1, -1):
                if self.board[y, self.tile_x + x] > 0:
                    # Calculate the y position for this column 
                    # if no other columns are taken into account.
                    cury = y + 1 - tile[x][0]
                    break
            if self.tile_y < cury:
                self.tile_y = cury

            if self.tile_y + np.max(tile) > self.rows:
                self.gameover = True
                dreward = -100
            else:
                # Change board entries at the newly placed tile to occupied.
                for x in range(len(tile)):
                    self.board[self.tile_y + tile[x][0]:self.tile_y + tile[x][1], 
                            x + self.tile_x] = 1

                # Remove full lines.
                removed_lines = 0
                for y in range(self.rows - 1, -1, -1):
                    if np.sum(self.board[y, :]) == self.cols:
                        removed_lines += 1
                        for y1 in range(y, self.rows - 1):
                            self.board[y1, :] = self.board[y1 + 1, :]
                        self.board[self.rows - 1, :] = Tetris.UNDEFINED
                
                dreward = 10 ** (removed_lines - 1) if removed_lines > 0 else 0
            
            self.next_tile()
            self.reward += dreward
            return dreward


In [ ]:
tetris = Tetris(rows=4, cols=4, max_tiles=50, random_seed=123456)

## Playing Tetris with the Command Line

In [ ]:
while not tetris.gameover:
    print(f"Tile {tetris.tile_count}/{tetris.max_tiles}")
    print(f"Reward: {tetris.reward}")
    print(f"Current tile {tetris.current_tile} with "
          f"orientation {tetris.tile_orientation} at position {tetris.tile_x}")
    print(tetris.TILES[tetris.current_tile][tetris.tile_orientation])
    print(tetris.board)
    
    cmd = input("Please enter your command (L, R, O, D, X): ").upper()
    print(f"Your input: {cmd}")
    
    if cmd == "L":
        tetris.move_left()
    elif cmd == "R":
        tetris.move_right()
    elif cmd == "O":
        tetris.rotate()
    elif cmd == "D":
        tetris.drop()
    elif cmd == "X":
        break

## Playing Tetris with a Graphical Interface with Pygame

In [ ]:
import pygame

def play_tetris_with_gui(tetris):
    """Play Tetris with GUI for human players."""
    TILE_SIZE = 20
    BLACK = (0, 0, 0)  # RGB code for black color.
    GREY = (128, 128, 128)  # RGB code for grey color.
    WHITE = (255, 255, 255)  # RGB code for white color.
    RED = (255, 0, 0)  # RGB code for red color.

    # Initialize the game engine.
    pygame.init()
    pygame.display.set_caption("TETRIS")
    screen = pygame.display.set_mode((200 + tetris.cols * TILE_SIZE, 
                                    200 + tetris.rows * TILE_SIZE))
    pygame.key.set_repeat(300, 100)  # Set keyboard delay and interval in ms.
    font = pygame.font.SysFont("Calibri", 25, True)

    # Loop until the window is closed.
    running = True
    while running:    
        # Paint game board.
        if pygame.display.get_active():
            screen.fill(WHITE)

            for i in range(tetris.rows):
                for j in range(tetris.cols):
                    pygame.draw.rect(
                        screen, 
                        GREY, 
                        [100 + TILE_SIZE * j, 
                         80 + TILE_SIZE * (tetris.rows - i), 
                         TILE_SIZE, 
                         TILE_SIZE], 
                        1
                    )
                    if tetris.board[i][j] > 0:
                        pygame.draw.rect(
                            screen, 
                            BLACK,
                            [101 + TILE_SIZE * j, 
                             81 + TILE_SIZE * (tetris.rows - i), 
                             TILE_SIZE - 2, 
                             TILE_SIZE - 2],
                        )
            
            tile = tetris.TILES[tetris.current_tile][tetris.tile_orientation]
            for x in range(len(tile)):
                for y in range(tile[x][0], tile[x][1]):
                    pygame.draw.rect(
                        screen,
                        RED,
                        [101 + TILE_SIZE * (x + tetris.tile_x), 
                         81 + TILE_SIZE * (tetris.rows - (y + tetris.tile_y)), 
                         TILE_SIZE - 2,
                         TILE_SIZE - 2]
                    )
            
            screen.blit(
                font.render(f"Reward: {tetris.reward}", True, BLACK), 
                [0, 0]
            )
            screen.blit(
                font.render(f"Tile {tetris.tile_count}/{tetris.max_tiles}", 
                            True, BLACK), 
                [0, 30]
            )
            if tetris.gameover:
                screen.blit(font.render("G A M E   O V E R", True, RED), 
                            [40, 100 + tetris.rows * TILE_SIZE])
                screen.blit(font.render("Press ESC to try again", True, RED), 
                            [10, 100 + tetris.rows * TILE_SIZE + 30])

        pygame.display.flip()
        
        # Get user input.
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    tetris.restart()
                if not tetris.gameover:
                    if event.key == pygame.K_LEFT:
                        tetris.move_left()
                    elif event.key == pygame.K_RIGHT:
                        tetris.move_right()
                    elif event.key == pygame.K_UP:
                        tetris.rotate()
                    elif event.key == pygame.K_DOWN:
                        tetris.drop()    
                
    pygame.quit()

In [ ]:
play_tetris_with_gui(tetris.restart())